In [1]:
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch
import torch.optim as optim

# Dataset directory
data_dir = r"C:\Users\egese\Desktop\dataset\mapped_train"

# Transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Load dataset
full_dataset = datasets.ImageFolder(data_dir, transform=train_transform)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
val_dataset.dataset.transform = val_transform

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.efficientnet_b0(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(full_dataset.classes))
model = model.to(device)

# Training setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
num_epochs = 50  # max sınır koyuyoruz
patience = 3     # kaç epoch sabit kalırsa dursun
delta = 0.001    # %0.1'lik gelişme sınırı

best_val_acc = 0.0
no_improve_epochs = 0
training_log = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)

    # Validation
    model.eval()
    correct = 0
    total = 0
    val_loss = 0.0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = correct / total

    training_log.append({
        "epoch": epoch + 1,
        "train_loss": avg_train_loss,
        "val_loss": avg_val_loss,
        "val_accuracy": val_accuracy
    })

    print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}, Val Accuracy = {val_accuracy:.2%}")

    # Early stopping condition
    if val_accuracy - best_val_acc < delta:
        no_improve_epochs += 1
        print(f"⚠️  No significant improvement for {no_improve_epochs} epoch(s).")
        if no_improve_epochs >= patience:
            print("!!!Early stopping triggered.")
            break
    else:
        best_val_acc = val_accuracy
        no_improve_epochs = 0

# Save final model
torch.save(model.state_dict(), "efficientnet_b0_best.pth")

C:\ProgramData\anaconda3\anaconda3 2023.07\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\ProgramData\anaconda3\anaconda3 2023.07\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1: Train Loss = 1.6116, Val Loss = 1.4781, Val Accuracy = 58.25%
Epoch 2: Train Loss = 1.1505, Val Loss = 0.9655, Val Accuracy = 72.82%
Epoch 3: Train Loss = 0.8166, Val Loss = 0.6920, Val Accuracy = 77.67%
Epoch 4: Train Loss = 0.5842, Val Loss = 0.5270, Val Accuracy = 80.58%
Epoch 5: Train Loss = 0.4021, Val Loss = 0.4086, Val Accuracy = 87.38%
Epoch 6: Train Loss = 0.2558, Val Loss = 0.2998, Val Accuracy = 91.26%
Epoch 7: Train Loss = 0.1764, Val Loss = 0.2188, Val Accuracy = 93.20%
Epoch 8: Train Loss = 0.1101, Val Loss = 0.1743, Val Accuracy = 93.20%
⚠️  No significant improvement for 1 epoch(s).
Epoch 9: Train Loss = 0.0832, Val Loss = 0.1569, Val Accuracy = 93.20%
⚠️  No significant improvement for 2 epoch(s).
Epoch 10: Train Loss = 0.0560, Val Loss = 0.1514, Val Accuracy = 94.17%
Epoch 11: Train Loss = 0.0439, Val Loss = 0.1455, Val Accuracy = 94.17%
⚠️  No significant improvement for 1 epoch(s).
Epoch 12: Train Loss = 0.0506, Val Loss = 0.1475, Val Accuracy = 93.20%
⚠️  

ModuleNotFoundError: No module named 'ace_tools'

In [3]:
import pandas as pd

# model success 
df = pd.DataFrame(training_log)
display(df)

,epoch,train_loss,val_loss,val_accuracy
0,1,1.611616,1.478098,0.582524
1,2,1.150481,0.965462,0.728155
2,3,0.816571,0.692018,0.776699
3,4,0.584209,0.527017,0.805825
4,5,0.402105,0.408629,0.873786
5,6,0.255826,0.299823,0.912621
6,7,0.176435,0.218818,0.932039
7,8,0.110119,0.174252,0.932039
8,9,0.083166,0.156869,0.932039
9,10,0.055984,0.151421,0.941748
